In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler


warnings.filterwarnings("ignore")



## Load data

In [2]:
data = pd.read_csv('data/BAG_OR_long.csv', index_col=0)

In [3]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


# Odds ratios

## Without co-vars

In [4]:
results_merge_df_all = data.copy()

results_merge_df_all = results_merge_df_all.loc[:, ~results_merge_df_all.columns.duplicated()]

df_directions_odd = pd.DataFrame()

for i in vars_:
    
    y_ols = results_merge_df_all['GAP_bin']

    
    X_ols = results_merge_df_all[[i] + ['delta_time']]
    X_train, X_test, y_train, y_test = train_test_split(X_ols, y_ols, test_size=0.2, random_state=42)
    

    scaler = MinMaxScaler((0.05, 0.95))
    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    X_train_scaled['intercept'] = 1
    X_test_scaled['intercept'] = 1
    
    model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit()
    
    params = model.params
    conf = np.exp(model.conf_int())
    conf['RR'] = np.exp(params)
    conf['z'] =model.tvalues
    conf['P>|z|'] =model.pvalues
    conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']
    
    df = conf.loc[i:i]
    df['Feature'] = i
    df_directions_odd = pd.concat([df_directions_odd, df])

df_directions_odd = df_directions_odd.reset_index(drop=True)


## Odd ratios with macrosocials

In [5]:
vars_social_physical = ['gender_equal_l', 'Polution_conc_inv', 'Eq']

vars_sociopolitical = ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est']

In [6]:
results_merge_df_all = data.copy()



count = 1
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    df_directions_odd = pd.DataFrame()
    df_directions_cov_odd = pd.DataFrame()
    for i in vars_:
        
        y_ols = results_merge_df_all['GAP_bin']
    
        
        X_ols = results_merge_df_all[[i] + [covar] + ['delta_time']]
        X_train, X_test, y_train, y_test = train_test_split(X_ols, y_ols, test_size=0.2, random_state=42)
        

        scaler = MinMaxScaler((0.05, 0.95))
        X_train_scaled = scaler.fit_transform(X_train)
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
        
        X_test_scaled = scaler.transform(X_test)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
        
        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1
        
        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit()
        
        params = model.params
        conf = np.exp(model.conf_int())
        conf['RR'] = np.exp(params)
        conf['z'] =model.tvalues
        conf['P>|z|'] =model.pvalues
        conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']
        
        df = conf.loc[i:i]
        df['Feature'] = i
        df_directions_odd = pd.concat([df_directions_odd, df])

        
        df_covar = conf.loc[covar:covar]
        df_covar['Feature'] = i + '('+covar + ')'
        df_directions_cov_odd = pd.concat([df_directions_cov_odd, df_covar])
        
    df_directions_odd = df_directions_odd.reset_index(drop=True)
    df_directions_cov_odd = df_directions_cov_odd.reset_index(drop=True)
    
